In [1]:
!git clone https://github.com/TheSoundOfAIOSR/rg_sound_generation.git
%cd rg_sound_generation

fatal: destination path 'rg_sound_generation' already exists and is not an empty directory.
/content/rg_sound_generation


In [2]:
!pip install -q -r requirements_colab.txt

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import tensorflow as tf
import tsms
from tcae.model import MtVae
from tcae.localconfig import LocalConfig
from tcae.dataset import get_dataset
from tcae.train import get_zero_batch, validation_step
from IPython.display import Audio


conf = LocalConfig()

In [5]:
conf.load_config_from_file("deployed/conf.json")
conf.dataset_dir = "/content/drive/MyDrive/the_sound_of_ai/new_data"
conf.batch_size = 1

In [6]:
model = MtVae(conf)
_ = model(get_zero_batch(conf))
model.load_weights("deployed/model.h5")

_, _, test_dataset = get_dataset(conf)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [7]:
batch = next(iter(test_dataset))

losses = validation_step(model, batch)

print(losses)

{'loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.009195656>, 'f0_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.0005076269>, 'h_freq_shifts_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.00021114427>, 'mag_env_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.00032668098>, 'h_mag_dist_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.0028061662>, 'h_mag_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.0053440374>, 'no_mask_h_mag_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.041198425>, 'h_phase_diff_loss': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>}


In [8]:
preds = model.predict(batch)
transformed = conf.data_handler.prediction_transform(preds)

note_number = batch["note_number"]
note_number = tf.argmax(note_number, axis=-1) + conf.starting_midi_pitch
note_number = note_number[:, tf.newaxis, tf.newaxis]

h_freq, h_mag, h_phase = conf.data_handler.denormalize(transformed, batch["mask"], note_number)

audio = tsms.core.harmonic_synthesis(h_freq, h_mag, h_phase, conf.sample_rate, conf.frame_size)

In [9]:
Audio(audio[0], rate=conf.sample_rate)